In [7]:
from dotenv import load_dotenv
import oracledb
import cx_Oracle
import os
import openai

load_dotenv()

True

# Generating SQL query equivalent to query in natural language through ChatGPT

This approach consists of generating the SQL query equivalent to the natural language query in ChatGPT.
In order to verify that the sql generated by ChatGPT was valid, it connected to the oracle database and ran the queries.


### Conectando ao Oracle Database
https://medium.com/oracledevs/using-the-development-branch-of-sqlalchemy-2-0-with-python-oracledb-d6e89090899c
https://stackoverflow.com/questions/14140902/using-oracle-service-names-with-sqlalchemy

In [8]:
username = str(os.getenv('DB_USER_NAME'))
passwd = str(os.getenv('DB_PASS'))
hostname = str(os.getenv('DB_HOST'))
port = str(os.getenv('DB_PORT'))
database = str(os.getenv('DB_NAME'))
sqldriver = str(os.getenv('SQL_DRIVER'))
servicename = str(os.getenv('SERVICE_NAME'))

# dsn = """(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={hostname})
#     (PORT={port}))(CONNECT_DATA=(SERVER=DEDICATED)(SERVICE_NAME={servicename})))"""

print("ok")
dsnStr = cx_Oracle.makedsn(host = hostname, port = port, service_name = servicename).replace("SID", "SERVICE_NAME")    
conn = oracledb.connect(user=username, password=passwd,
                              dsn=dsnStr)


ok


#### Testando a conexão com uma consulta simples

In [9]:
def execute_sql(sql):
    if sql.endswith(";"):
        sql = sql[:-1]
    try:
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            print(row)
        cursor.close()
    except: 
        print("SQL Error")
    pass

execute_sql("SELECT * FROM MONDIAL_TMDC;")



('http://www.semwebtech.org/mondial/AIRPORT', 'Airport', None)
('http://www.semwebtech.org/mondial/BORDERS', 'Borders', None)
('http://www.semwebtech.org/mondial/CITY', 'City', None)
('http://www.semwebtech.org/mondial/CITYOTHERNAME', 'Other Name City', None)
('http://www.semwebtech.org/mondial/CITYPOPS', 'Population City', None)
('http://www.semwebtech.org/mondial/CONTINENT', 'Continent', None)
('http://www.semwebtech.org/mondial/COUNTRY', 'Country', None)
('http://www.semwebtech.org/mondial/COUNTRYPOPS', 'Population Country', None)
('http://www.semwebtech.org/mondial/DESERT', 'Desert', None)
('http://www.semwebtech.org/mondial/ECONOMY', 'Economy', None)
('http://www.semwebtech.org/mondial/ENCOMPASSES', 'Encompasses', None)
('http://www.semwebtech.org/mondial/ETHNICGROUP', 'Ethnic Group', None)
('http://www.semwebtech.org/mondial/GEO_DESERT', 'Geo Desert', None)
('http://www.semwebtech.org/mondial/GEO_MOUNTAIN', 'Geo Mountain', None)
('http://www.semwebtech.org/mondial/ISLAND', 'Islan

### Criando a classe modelo do ChatGPT

In [10]:
openai.api_key = os.getenv("OPENAI_API_KEY")

class ChatModel:
    def __init__(self, prompt):
        self.messages = []
        self.messages.append({"role": "system", "content": prompt})
    
    def get_response(self, input):
        self.messages.append({"role": "user", "content": input})
        response =  openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages= self.messages,
            temperature=0.6,
            max_tokens=400
        )
        return response["choices"][0]["message"]["content"].strip()

### Carregando o prompt e criando um modelo

In [11]:
prompt = "You are a helpful assistant that generates the SQL equivalent of a natural language query. Generate SQL query from this text:" 
print(prompt)
chat = ChatModel(prompt)

You are a helpful assistant that generates the SQL equivalent of a natural language query. Generate SQL query from this text:


### Executando Testes

In [12]:
response = chat.get_response("What are the provinces with an area greater than 1000?")
print(response)
execute_sql(response)

SELECT province_name
FROM provinces
WHERE area > 1000;
SQL Error


In [13]:
response = chat.get_response("What are the provinces with an area more than 1000?")
print(response)
execute_sql(response)

SELECT * FROM provinces WHERE area > 1000;
SQL Error


In [40]:
response = chat.get_response("Tell me about cities of Cuba")
print(response)
execute_sql(response)

To retrieve information about cities in Cuba, you can use the following SQL query:

SELECT * FROM cities WHERE country = 'Cuba';

This query will retrieve all the cities from the "cities" table where the country is set to "Cuba". Replace "cities" with the actual name of the table that contains the city data in your database schema.
SQL Error


In [15]:
response = chat.get_response("city of Cuba")
print(response)
execute_sql(response)

To generate a SQL query for your request, I need to know the structure of the database schema. Can you provide me with the table names and columns related to the provinces and their areas?
SQL Error


In [41]:
response = chat.get_response("Say me about the total of city of cuba?")
print(response)
execute_sql(response)

The total number of cities in Cuba is 168.
SQL Error


In [42]:
response = chat.get_response("Say me about the total of Cities of cuba?")
print(response)
execute_sql(response)

The total number of cities in Cuba is approximately 168.
SQL Error


In [18]:
response = chat.get_response("Say me about the total of cities of cuba?")
print(response)
execute_sql(response)

SELECT COUNT(*) AS total_cities
FROM cities
WHERE country = 'Cuba';
SQL Error


In [47]:
response = chat.get_response("Tell me the largest area.")
print(response)
execute_sql(response)

To generate the SQL query, I would need to know the specific table and column names in the database where the information is stored.
SQL Error


In [51]:
response = chat.get_response("Areas different from 100")
print(response)
execute_sql(response)

To generate the SQL query, I need to know the structure of the database table that contains the information about the provinces. Can you provide me with the table schema or the names of the relevant columns?
SQL Error


In [21]:
response = chat.get_response("Shows me cities with longitude between 0 and 6")
print(response)
execute_sql(response)

SELECT * FROM cities WHERE longitude BETWEEN 0 AND 6;
SQL Error


In [22]:
response = chat.get_response("Shows me Airports with elevation more than 5000 ")
print(response)
execute_sql(response)

SELECT *
FROM Airports
WHERE elevation > 5000;
SQL Error


In [23]:
response = chat.get_response("What language is spoken in Poland?")
print(response)
execute_sql(response)

SELECT language 
FROM countries 
WHERE country = 'Poland'
SQL Error


In [54]:
response = chat.get_response("Tell me about the religion that is practiced in Haiti")
print(response)
execute_sql(response)

The majority of the population in Haiti practices Christianity, with the largest denomination being Roman Catholicism. However, there is also a significant presence of Protestantism, particularly Baptist and Pentecostal denominations. Additionally, there are adherents of Vodou, which is a syncretic religion that combines elements of African traditional religions with Catholicism.
SQL Error


In [25]:
response = chat.get_response("How deep is Lake Kariba?")
print(response)
execute_sql(response)

SELECT depth
FROM lakes
WHERE name = 'Lake Kariba';
SQL Error


In [26]:
response = chat.get_response("What is the capital of Georgia?")
print(response)
execute_sql(response)

The capital of Georgia is Tbilisi.
SQL Error


In [27]:
response = chat.get_response("What is the total of provinces of Netherlands?")
print(response)
execute_sql(response)

SELECT COUNT(*) AS total_provinces
FROM provinces
WHERE country = 'Netherlands';
SQL Error


In [28]:
response = chat.get_response("total provinces Netherland")
print(response)
execute_sql(response)

SELECT COUNT(*) AS total_provinces
FROM provinces;
SQL Error


In [29]:
response = chat.get_response("total Province Netherlands")
print(response)
execute_sql(response)

SELECT COUNT(*) AS total_provinces
FROM provinces
WHERE country = 'Netherlands';
SQL Error


In [30]:
response = chat.get_response("What percentage of religious people are hindu in thailand?")
print(response)
execute_sql(response)

SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM population)) AS percentage_hindu
FROM population
WHERE country = 'Thailand' AND religion = 'Hindu';
SQL Error


In [31]:
response = chat.get_response("What are the provinces with an area greater than 1000 in Niger?")
print(response)
execute_sql(response)

SELECT * 
FROM provinces
WHERE area > 1000
AND country = 'Niger';
SQL Error


In [32]:
response = chat.get_response("The Chatanga River belongs to which country?")
print(response)
execute_sql(response)

I'm sorry, but I'm not able to provide the SQL equivalent of the given text as it does not involve a SQL query.
SQL Error


In [33]:
response = chat.get_response("What province is the city of Glendale in?")
print(response)
execute_sql(response)

What is the population of Tokyo?
SQL Error


In [34]:
response = chat.get_response("What is the area of the city Granada in Nicaragua?")
print(response)
execute_sql(response)

SELECT area
FROM cities
WHERE name = 'Granada' AND country = 'Nicaragua';
SQL Error


In [38]:
response = chat.get_response("tell me about the Caribbean economic")
print(response)
execute_sql(response)

I'm sorry, but I'm unable to generate the SQL query for that request.
SQL Error


In [36]:
response = chat.get_response("Tell me about the country with the area largest.")
print(response)
execute_sql(response)

SELECT country_name
FROM countries
WHERE area = (SELECT MAX(area) FROM countries);
SQL Error


In [37]:
response = chat.get_response("What is the host province of the International Energy Agency?")
print(response)
execute_sql(response)

I'm sorry, but I couldn't find any information about a host province for the International Energy Agency. The International Energy Agency (IEA) is an autonomous agency that operates at an international level and does not have a specific host province.
SQL Error


In [39]:
response = chat.get_response("What is Mexico's independence day?")
print(response)
execute_sql(response)

Mexico's independence day is celebrated on September 16th.
SQL Error
